# OpenAI Embeddings

(C) 2024 by [Damir Cavar](http://damir.cavar.me/)

Pulling embeddings for words and phrases from OpenAI's GPT.

    Model	~ Pages per dollar	Performance on MTEB eval	Max input
    text-embedding-3-small	62,500	62.3%	8191
    text-embedding-3-large	9,615	64.6%	8191
    text-embedding-ada-002	12,500	61.0%	8191

For this code to work you will need the `openai` Python module.

In [ ]:
!pip install -U openai

In the file `secret.py` in the same folder as this notebook create the variable `openai_apikey` as a string variable and set it to your API key from OpenAI. You might have to create the file `secret.py`. 

In [8]:
import openai
from tenacity import retry, wait_random_exponential, stop_after_attempt, retry_if_not_exception_type
from secret import openai_apikey

Use either  `text-embedding-3-small` or `text-embedding-3-large` for the embeddings.

In [9]:
EMBEDDING_MODEL = "text-embedding-3-large"
EMBEDDING_CTX_LENGTH = 8191
EMBEDDING_ENCODING = 'cl100k_base'

Create a client for the OpenAI API endpoint:

In [10]:
client = openai.OpenAI(api_key=openai_apikey)

The following function will fetch the vectors for the words in the text_or_tokens string. It will wait and repeat, if the endpoint is busy or a delay is imposed by it:

In [11]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6), retry=retry_if_not_exception_type(openai.BadRequestError))
def get_embedding(text_or_tokens, client, model=EMBEDDING_MODEL):
	text_or_tokens = text_or_tokens.replace("\n", " ")
	return client.embeddings.create(input=text_or_tokens, model=model).data[0].embedding

Use this word list:

In [12]:
wordlist = list(set("""
cat dog fish bird
car truck bike bus
""".split()))

Loop over the word list and request the embedding vector from the OpenAI API endpoint:

In [13]:
for word in wordlist:
    try:
        embeddings = get_embedding(word, client)
        print(word, embeddings)
    except openai.BadRequestError as e:
        print(e)

car [-0.021500570699572563, 0.037350136786699295, 0.007882985286414623, 0.00410868227481842, -0.005187054164707661, 0.01832396909594536, -0.027736738324165344, -0.004639508668333292, -0.014127513393759727, 0.008146308362483978, -0.013592506758868694, -0.008497406728565693, -0.010833879932761192, -0.03405650332570076, -0.03470854088664055, 0.0020062739495187998, -0.055440038442611694, -0.00782028865069151, 0.014796270988881588, -0.017989590764045715, 0.03260195255279541, -0.013433676213026047, 0.011795219965279102, 0.027970803901553154, 0.04246613383293152, 0.033671967685222626, 0.007490089628845453, 0.030227862298488617, -0.04848495498299599, -0.02758626826107502, 0.011410683393478394, 0.05029060319066048, -0.014144232496619225, 0.026265470311045647, -0.034608226269483566, 0.028539247810840607, -0.00164368178229779, -0.002386630279943347, -0.008777448907494545, 0.005425299517810345, -0.0012257080525159836, -0.010282154195010662, -0.028773313388228416, 0.0028526708483695984, 0.014102434

(C) 2024 by [Damir Cavar](http://damir.cavar.com/)